# Install Dependencies

In [ ]:
!pip install llama-index datasets llama-index-callbacks-arize-phoenix arize-phoenix llama-index-vector-stores-chroma llama-index-llms-ollama llama-index-embeddings-ollama llama-index-tools-google pip install llama-index-tools-mcp -U -q

# Creating a FunctionTool

In [1]:
from llama_index.core.tools import FunctionTool


def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is ☀️"


tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location.",
)
tool.call("Boston")

Getting weather for Boston


ToolOutput(blocks=[TextBlock(block_type='text', text='The weather in Boston is ☀️')], tool_name='my_weather_tool', raw_input={'args': ('Boston',), 'kwargs': {}}, raw_output='The weather in Boston is ☀️', is_error=False)

# Creating a QueryEngineTool

In [2]:
# Load and prepare the persona dataset
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

/opt/anaconda3/envs/hf_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load documents
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
print(f"Loaded {len(documents)} documents")

Loaded 5000 documents


In [4]:
import chromadb


from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# Use Ollama for embeddings and LLM
llm = Ollama(model="llama3:8b")
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# Initialize ChromaDB
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=50),
        embed_model,
    ],
    vector_store=vector_store,
)


In [5]:
from llama_index.core import VectorStoreIndex


# Create index from vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, 
    embed_model=embed_model
)

# Create query engine
query_engine = index.as_query_engine(llm=llm)

# Create tool
tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="persona_search",
    description="Search through party attendee personas to find relevant information",
)

# Query the tool
response = await tool.acall(
    "which personas are in AI and technology?"
)
print(response)

The only persona mentioned is Persona 105.


# Creating Toolspecs

In [6]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
tool_spec_list

In [7]:
[print(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

load_data load_data() -> List[llama_index.core.schema.Document]
Load emails from the user's account.
search_messages search_messages(query: str, max_results: Optional[int] = None)

        Searches email messages given a query string and the maximum number
        of results requested by the user
           Returns: List of relevant message objects up to the maximum number of results.

        Args:
            query (str): The user's query
            max_results (Optional[int]): The maximum number of search results
            to return.

        
create_draft create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str

        Create and insert a draft email.
           Print the returned draft's message and id.
           Returns: Draft object, including draft id and message meta data.

        Args:
            to (Optional[str]): The email addresses to send the message to
            subject (Optional[str]): The subject for th

[None, None, None, None, None, None]